In [1]:
import requests
import base64
import json
import pprint
import pandas as pd
from datetime import datetime
import logging

from stubhub_params import (
    CONSUMER_KEY,
    CONSUMER_SECRET,
    STUBHUB_USERNAME,
    STUBHUB_PASSWORD
)

pd.options.display.max_columns = 100

In [2]:
combo = CONSUMER_KEY + ':' + CONSUMER_SECRET
basic_authorization_token = base64.b64encode(combo.encode('utf-8'))

In [4]:
## Generate access token

headers = {
    'Authorization': f'Basic {basic_authorization_token.decode("utf-8")}',
    'Content-Type': 'application/json',
}

params = (
    ('grant_type', 'password'),
)

data = json.dumps(
    {"username": STUBHUB_USERNAME,
     "password": STUBHUB_PASSWORD}
)
url = 'https://api.stubhub.com/sellers/oauth/accesstoken'
response = requests.post(url, headers=headers, params=params, data=data)



In [5]:
access_token = response.json()['access_token']

In [6]:
headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
}

params = (
    ('name', 'Hamilton New York'),
    ('rows','500')
)

response = requests.get('https://api.stubhub.com/sellers/search/events/v3', headers=headers, params=params)

### Get Date and ID of events

In [7]:
hmltn_dts = pd.DataFrame(response.json()['events'])[['eventDateLocal','id']]

In [8]:
hmltn_dts['date_time'] = pd.to_datetime(hmltn_dts['eventDateLocal'].str[:19])
hmltn_dts['day_of_week']  = hmltn_dts['date_time'].dt.day_name()
hmltn_dts['time'] = hmltn_dts['date_time'].dt.hour
hmltn_dts = hmltn_dts.set_index('id')

In [9]:
hmltn_dts.head()

,eventDateLocal,date_time,day_of_week,time
id,,,,
104174354,2020-01-08T14:00:00-0500,2020-01-08 14:00:00,Wednesday,14
104174375,2020-02-01T14:00:00-0500,2020-02-01 14:00:00,Saturday,14
104444563,2019-12-14T14:01:00-0500,2019-12-14 14:01:00,Saturday,14
104174386,2020-02-11T19:00:00-0500,2020-02-11 19:00:00,Tuesday,19
104174384,2020-02-08T20:00:00-0500,2020-02-08 20:00:00,Saturday,20


In [10]:
ids_of_int = hmltn_dts[((hmltn_dts['day_of_week']=='Friday')|
                       (hmltn_dts['day_of_week']=='Saturday'))
                    &(hmltn_dts['time']==20)].index.values

In [11]:
headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
}

params = (
    ('eventId', '104174936'),
    ('quantity', '>1'),
    ('rows','1000')
)

response = requests.get('https://api.stubhub.com/sellers/find/listings/v3/', headers=headers, params=params)

In [12]:
def get_rel_data(response, dt_event):
    '''
    takes response and extracts relevant data
    '''
    listings = pd.DataFrame(response.json()['listings'])
    listings['price'] = listings['pricePerProduct'].apply(lambda x: x['amount'])
    cols_interest = ['listingId','quantity','row','sectionName','zone','price']
    if set(cols_interest).issubset(listings.columns):
        return listings[cols_interest].assign(
            dt_event = dt_event,
            dt_pull = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        )
    else:
        return pd.DataFrame()

In [18]:
all_listings = pd.DataFrame()
problem_ids = []

for ii, id_curr in enumerate(ids_of_int):
    dt_event = hmltn_dts.loc[id_curr]['date_time']
    params = (
        ('eventId', str(id_curr)),
        ('quantity', '>1'),
        ('rows','1000')
    )
    response = requests.get('https://api.stubhub.com/sellers/find/listings/v3/', headers=headers, params=params)
    #check to make sure there are listings to look at
    if response.json()['totalListings']>0:
        listings_curr = get_rel_data(response, dt_event)
        if listings_curr.shape[0]>0:
            all_listings = all_listings.append(listings_curr).reset_index(drop=True)
        else:
            problem_ids.append(id_curr)
    else:
        problem_ids.append(id_curr)
    print(ii)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [21]:
#sample data
all_listings[(all_listings['zone']=='Mezzanine Front')
             &(all_listings['row']<'D')
            ].sort_values('dt_event').head()

,listingId,quantity,row,sectionName,zone,price,dt_event,dt_pull
4509,1539423882,2,A,Mezzanine Front Center,Mezzanine Front,750.0,2019-11-22 20:00:00,2019-11-17 10:46:06
4899,1539426449,2,A,Mezzanine Front Center,Mezzanine Front,900.0,2019-11-23 20:00:00,2019-11-17 10:46:07
4892,1539426454,2,B,Mezzanine Front Center,Mezzanine Front,750.0,2019-11-23 20:00:00,2019-11-17 10:46:07
3347,1473461794,2,C,Mezzanine Front - Left and Right,Mezzanine Front,437.0,2019-11-29 20:00:00,2019-11-17 10:46:00
3381,1492641546,2,B,Mezzanine Front Center,Mezzanine Front,638.0,2019-11-29 20:00:00,2019-11-17 10:46:00


In [22]:
id_curr

104447347

In [23]:
hmltn_dts[((hmltn_dts['day_of_week']=='Friday')|
                       (hmltn_dts['day_of_week']=='Saturday'))
                    &(hmltn_dts['time']==20)].reset_index().reset_index().set_index('id').loc[id_curr]

index                                  115
eventDateLocal    2020-02-14T20:01:00-0500
date_time              2020-02-14 20:01:00
day_of_week                         Friday
time                                    20
Name: 104447347, dtype: object